## DeepExplain - Toy example of a model with multiple inputs
### Keras with Functional API

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tempfile, sys, os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D, Concatenate
from keras import backend as K

# Import DeepExplain
from deepexplain.tensorflow import DeepExplain

Using TensorFlow backend.


In [4]:
# Define two simple numerical inputs
_x1 = np.array([[1]])
_x2 = np.array([[2]])

# Define model
# Here we implement y = sigmoid([w1*x1|w2*x2] * w3)
def init_model():
    x1 = Input(shape=(1,))
    x2 = Input(shape=(1,))

    t1 = Dense(1)(x1)
    t2 = Dense(1)(x2)
    t3 = Concatenate()([t1, t2])
    t4 = Dense(1)(t3)
    y = Activation('sigmoid')(t4)
    
    model = Model(inputs=[x1, x2], outputs=y)
    model.compile(optimizer='rmsprop', loss='mse')
    return model


model = init_model()
# This is a toy example. The random weight initialization will do just fine.
# model.fit(...)

# Make sure the model works
print ("Output: ", model.predict(x=[_x1, _x2]))

with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    # Need to reconstruct the graph in DeepExplain context, using the same weights.
    input_tensors = model.inputs
    fModel = Model(inputs = input_tensors, outputs = model.outputs)
    target_tensor = fModel(input_tensors)

    attributions = de.explain('grad*input', target_tensor, input_tensors, [_x1, _x2])
    print ("Attributions:\n", attributions)

Output:  [[ 0.98570454]]
DeepExplain: running "grad*input" explanation method (2)
Attributions:
 [array([[ 0.0108474]], dtype=float32), array([[ 0.04880605]], dtype=float32)]
